In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
#import ta
import cProfile
#import yfinance as yf
#import tensorflow as tf
#import tensorflow.compat.v2.feature_column as fc
from IPython.display import clear_output
import pyodbc

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing
#from utils.models import make_input_fn
#from utils.models import performanceTesting,scoreFunction

In [6]:
#!pip install ta

## Let's extract some data now!

In [7]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5' # 't:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200501']).astype(int)#,'20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = ['GOOG','TSLA']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

# run load_data()
# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, False)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

In [8]:
quoteData

NameError: name 'quoteData' is not defined

## Optimizing the data loader

In [181]:
# Function to clean the unpacked data from the compressed files.
def strList(ls):
    return list(map(lambda x: x.decode('utf-8'),ls))

# The following function is based on the research of (Lunde, 2016), summarized in the slides found here:
# https://econ.au.dk/fileadmin/site_files/filer_oekonomi/subsites/creates/Diverse_2016/PhD_High-Frequency/HF_TrQuData_v01.pdf

def formatDate(date,timestamps):
    return list(map(lambda x: date[0:4]+'/'+date[4:6]+'/'+date[6:]+' '+str(datetime.timedelta(seconds = int(str(x)[0:5]),
                                                     microseconds = int(str(x)[5:11]))),timestamps))


def load_data_v1(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]

    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                #print(ticker in QIC)
                # Getting the specific ticker information
#                 print(np.where(ticker==QIC))
#                 print(QI[5255])
                #print(QI[QIC==ticker])
                
                tickerInfo = QI[QIC==ticker][0]
                #print(tickerInfo)
                print('6 Lap time: %.3f' % ((time.time()-start)))
                
                
#                 if (verbose) & (i == 0):
#                         print('Ticker Information: ',tickerInfo,'\n')

                # Raw data
#                 tempData = raw_data['Quotes'][list(np.arange(tickerInfo[1],tickerInfo[1]+tickerInfo[2]))]
#                 print(tickerInfo[1],tickerInfo[2])
#                 print('Datatype: ',type(raw_data['Quotes']))
#                 print(raw_data['Quotes'].dtype)
#                 tempData = raw_data['Quotes'][tickerInfo[1]:(tickerInfo[1]+tickerInfo[2])]
#                 tempData = raw_data['Quotes'][403024728:600343]
#                 print(raw_data['Quotes'][[0,1,2,3,4,5]])
#                 print(raw_data['Quotes'][0:5])

                # Read direct
#                 print(raw_data['Quotes'][0])
#                 ll = len(raw_data['Quotes'][0])
#                 print(type(raw_data['Quotes']))
#                 print(type(raw_data['Quotes'][0]))
#                 arr = np.zeros((10,))#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 print(arr)
# #                 print(np.s_[0:10])
#                 raw_data['Quotes'].read_direct(arr,
#                                                np.s_[0:10],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
#                                                np.s_[0:10])#tickerInfo[2]
#                 print(arr[0:5])
        
                ## New proposal
            
#                 store = pd.HDFStore(path+'/'+file, 'r') #.get('Quotes')#'taqquote_20200401.h5'
#                 print(store is None)
#                 meta = store.select('Quotes', start=1, stop=1)
#                 dtDict = [(i,j) for i,j in zip(meta.columns,meta.dtypes)]
#                 dt2 = np.dtype(dtDict)
                
                #testDset = raw_data['Quotes']
                #test = testDset[testDset[0]]
#                 print(dt2)
                arr = np.zeros((tickerInfo[2].astype(int)+1,),dtype=dt2)#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 tickerInfo[1] = 0
#                 tickerInfo[2] = 500000
#                 print(np.s_[0:10])
                #print(int(tickerInfo[1]+tickerInfo[2]+1))
                #print(tickerInfo[1])
                #print(tickerInfo[1]+tickerInfo[2])
                raw_data['Quotes'].read_direct(arr,
                                               np.s_[tickerInfo[1]:int(tickerInfo[1]+tickerInfo[2]+1)],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
                                               np.s_[0:int(tickerInfo[2]+1)])#tickerInfo[2]
#                 print(arr)
#                 tempData = pd.DataFrame(arr,columns=dt2.names)
                print('7 Lap time: %.3f' % ((time.time()-start)))
                
                # For first file and first ticker.
                if (i == 0) & (j == 0):

#                     quoteData = pd.DataFrame(tempData, columns= tempData.dtype.names)
                    quoteData = pd.DataFrame(arr, columns= dt2.names)
                    # We remove all unnecessary variables
                    unnecessaryVariables = ['NationalBBOInd',
                                            'FinraBBOInd',
                                            'FinraQuoteIndicator',
                                            'SequenceNumber',
                                            'FinraAdfMpidIndicator',
                                            'QuoteCancelCorrection',
                                            'SourceQuote',
                                            'RPI',
                                            'ShortSaleRestrictionIndicator',
                                            'LuldBBOIndicator',
                                            'SIPGeneratedMessageIdent',
                                            'NationalBBOLuldIndicator',
                                            'ParticipantTimestamp',
                                            'FinraTimestamp',
                                            'FinraQuoteIndicator',
                                            'SecurityStatusIndicator']

                    quoteData = quoteData.drop(columns=unnecessaryVariables)
                    
                    print('8 Lap time: %.3f' % ((time.time()-start)))
                    
                    
                    quoteData.loc[:,'ex'] = strList(quoteData.ex)
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding the date of the file to the dataframe.
                    quoteData['Date'] = re.split('[._]',
                                                 file)[1]
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding a more readable timestamp - TEST IT
                    quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                                file)[1],
                                                                       quoteData.utcsec))
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                        quoteData.utcsec))
                    
                    print('13 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['Hour'] = quoteData.Timestamp.dt.hour
                    quoteData['Minute'] = quoteData.Timestamp.dt.minute
                    # Adding the ticker
                    quoteData['Ticker'] = ticker

                    if (verbose) & (i==0) & (j==0):
                        print('Sneak peak of the data\n\n',quoteData.head())
                    
                    #raw_data.close()
                else:

                    # Storing the data on the following tickers in a temporary variable.

                    temp = pd.DataFrame(arr, columns= dt2.names)
                    # Removing all unnecessary variables
                    temp = temp.drop(columns=unnecessaryVariables)

                    temp.loc[:,'ex'] = strList(temp.ex)
                    temp.loc[:,'mode'] = strList(temp['mode'])

                    # Adding the date of the file to the dataframe.
                    temp['Date'] = re.split('[._]',file)[1]

                    # Adding a more readable timestamp - TEST IT
                    temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                    temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                    temp['Hour'] = temp.Timestamp.dt.hour
                    temp['Minute'] = temp.Timestamp.dt.minute

                    # Adding the ticker
                    temp['Ticker'] = ticker

                    # Adding the new data
                    quoteData = pd.concat([quoteData,temp])

    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v2(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
                print(tickerInfo)
            print('6 Lap time: %.3f' % ((time.time()-start)))

            # Raw data
            data_range = np.concatenate([np.arange(i,j+i) for i,j in zip(pos_start,pos_range)])
            tempData = raw_data['Quotes'][data_range]
                       
            print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                dates = pd.to_datetime(quoteData.loc[:,'Date'], format='%Y%m%d', errors='ignore')
                times = pd.to_timedelta(quoteData.loc[:,'utcsec'])
#                 datetimes  = dates + times
#                 datetimes
                quoteData['Timestamp'] = dates + times
#                 quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
#                                                                             file)[1],
#                                                                    quoteData.utcsec))
                print('12 Lap time: %.3f' % ((time.time()-start)))

#                 quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
#                                                     quoteData.utcsec))

                print('13 Lap time: %.3f' % ((time.time()-start)))

                quoteData['Hour'] = quoteData.Timestamp.dt.hour
                quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= d2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
                temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                
                print('12 Lap time: %.3f' % ((time.time()-start)))
                
                temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                temp['Hour'] = temp.Timestamp.dt.hour
                temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                temp['Ticker'] = ticker

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v3(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]

    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            
            store = pd.HDFStore(path+'/'+file, 'r')
            
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]

                print('6 Lap time: %.3f' % ((time.time()-start)))
                
                meta = store.select('Quotes', start=tickerInfo[1], stop=(tickerInfo[1]+tickerInfo[2]))

                print('7 Lap time: %.3f' % ((time.time()-start)))
                
                # For first file and first ticker.
                if (i == 0) & (j == 0):

#                     quoteData = pd.DataFrame(tempData, columns= tempData.dtype.names)
                    quoteData = meta.copy(deep=True)#pd.DataFrame(arr, columns= dt2.names)
                    # We remove all unnecessary variables
                    unnecessaryVariables = ['NationalBBOInd',
                                            'FinraBBOInd',
                                            'FinraQuoteIndicator',
                                            'SequenceNumber',
                                            'FinraAdfMpidIndicator',
                                            'QuoteCancelCorrection',
                                            'SourceQuote',
                                            'RPI',
                                            'ShortSaleRestrictionIndicator',
                                            'LuldBBOIndicator',
                                            'SIPGeneratedMessageIdent',
                                            'NationalBBOLuldIndicator',
                                            'ParticipantTimestamp',
                                            'FinraTimestamp',
                                            'FinraQuoteIndicator',
                                            'SecurityStatusIndicator']

                    quoteData = quoteData.drop(columns=unnecessaryVariables)
                    
                    print('8 Lap time: %.3f' % ((time.time()-start)))
                    
                    
#                     quoteData.loc[:,'ex'] = strList(quoteData.ex)
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                    
#                     quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding the date of the file to the dataframe.
                    quoteData['Date'] = re.split('[._]',
                                                 file)[1]
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding a more readable timestamp - TEST IT
                    quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                                file)[1],
                                                                       quoteData.utcsec))
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                        quoteData.utcsec))
                    
                    print('13 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['Hour'] = quoteData.Timestamp.dt.hour
                    quoteData['Minute'] = quoteData.Timestamp.dt.minute
                    # Adding the ticker
                    quoteData['Ticker'] = ticker

                    if (verbose) & (i==0) & (j==0):
                        print('Sneak peak of the data\n\n',quoteData.head())
                    
                    #raw_data.close()
                else:

                    # Storing the data on the following tickers in a temporary variable.

                    temp = meta.copy(deep=True)#pd.DataFrame(arr, columns= dt2.names)
                    # Removing all unnecessary variables
                    temp = temp.drop(columns=unnecessaryVariables)

#                     temp.loc[:,'ex'] = strList(temp.ex)
#                     temp.loc[:,'mode'] = strList(temp['mode'])

                    # Adding the date of the file to the dataframe.
                    temp['Date'] = re.split('[._]',file)[1]

                    # Adding a more readable timestamp - TEST IT
                    temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                    temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                    temp['Hour'] = temp.Timestamp.dt.hour
                    temp['Minute'] = temp.Timestamp.dt.minute

                    # Adding the ticker
                    temp['Ticker'] = ticker

                    # Adding the new data
                    quoteData = pd.concat([quoteData,temp])

    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v4(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
                print(tickerInfo)
            print('6 Lap time: %.3f' % ((time.time()-start)))

                # Raw data
            data_range = np.concatenate([np.arange(i,j+i) for i,j in zip(pos_start,pos_range)])
            tempData = raw_data['Quotes'][np.r_[data_range]]
            l = len(data_range)
            arr = np.zeros((l,),dtype=dt2)

            raw_data['Quotes'].read_direct(arr,
                                           np.s_[data_range],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
                                           np.s_[0:l])#tickerInfo[2]
            
            print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                            file)[1],
                                                                   quoteData.utcsec))
                print('12 Lap time: %.3f' % ((time.time()-start)))

                quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                    quoteData.utcsec))

                print('13 Lap time: %.3f' % ((time.time()-start)))

                quoteData['Hour'] = quoteData.Timestamp.dt.hour
                quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= d2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
                temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                
                print('12 Lap time: %.3f' % ((time.time()-start)))
                
                temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                temp['Hour'] = temp.Timestamp.dt.hour
                temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                temp['Ticker'] = ticker

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData


def load_data_v5(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
                print(tickerInfo)
            print('6 Lap time: %.3f' % ((time.time()-start)))

            # use boolean mask to slice all at once
            selector = zip(pos_start, pos_range) 
            mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)
            for (pos_start, pos_range) in selector:
                mask[pos_start : pos_start + pos_range] = True
            tempData = raw_data['Quotes'][mask]
            #print(tempData[:5])
            
            print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                dates = pd.to_datetime(quoteData.loc[:,'Date'], format='%Y%m%d', errors='ignore')
                times = pd.to_timedelta(quoteData.loc[:,'utcsec'])
#                 datetimes  = dates + times
#                 datetimes
                quoteData['Timestamp'] = dates + times
#                 quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
#                                                                             file)[1],
#                                                                    quoteData.utcsec))
                print('12 Lap time: %.3f' % ((time.time()-start)))

#                 quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
#                                                     quoteData.utcsec))

                print('13 Lap time: %.3f' % ((time.time()-start)))

                quoteData['Hour'] = quoteData.Timestamp.dt.hour
                quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= d2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
                temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                
                print('12 Lap time: %.3f' % ((time.time()-start)))
                
                temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                temp['Hour'] = temp.Timestamp.dt.hour
                temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                temp['Ticker'] = ticker

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData


# v6 is v1 (loop over tickers and read with h5py) with boolean masking instead of read_direct
def load_data_v6(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]

    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                #print(ticker in QIC)
                # Getting the specific ticker information
#                 print(np.where(ticker==QIC))
#                 print(QI[5255])
                #print(QI[QIC==ticker])
                
                tickerInfo = QI[QIC==ticker][0]
                #print(tickerInfo)
                print('6 Lap time: %.3f' % ((time.time()-start)))
                
                
#                 if (verbose) & (i == 0):
#                         print('Ticker Information: ',tickerInfo,'\n')

                # Raw data
#                 tempData = raw_data['Quotes'][list(np.arange(tickerInfo[1],tickerInfo[1]+tickerInfo[2]))]
#                 print(tickerInfo[1],tickerInfo[2])
#                 print('Datatype: ',type(raw_data['Quotes']))
#                 print(raw_data['Quotes'].dtype)
#                 tempData = raw_data['Quotes'][tickerInfo[1]:(tickerInfo[1]+tickerInfo[2])]
#                 tempData = raw_data['Quotes'][403024728:600343]
#                 print(raw_data['Quotes'][[0,1,2,3,4,5]])
#                 print(raw_data['Quotes'][0:5])

                # Read direct
#                 print(raw_data['Quotes'][0])
#                 ll = len(raw_data['Quotes'][0])
#                 print(type(raw_data['Quotes']))
#                 print(type(raw_data['Quotes'][0]))
#                 arr = np.zeros((10,))#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 print(arr)
# #                 print(np.s_[0:10])
#                 raw_data['Quotes'].read_direct(arr,
#                                                np.s_[0:10],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
#                                                np.s_[0:10])#tickerInfo[2]
#                 print(arr[0:5])
        
                ## New proposal
            
#                 store = pd.HDFStore(path+'/'+file, 'r') #.get('Quotes')#'taqquote_20200401.h5'
#                 print(store is None)
#                 meta = store.select('Quotes', start=1, stop=1)
#                 dtDict = [(i,j) for i,j in zip(meta.columns,meta.dtypes)]
#                 dt2 = np.dtype(dtDict)
                
                #testDset = raw_data['Quotes']
                #test = testDset[testDset[0]]
#                 print(dt2)
                arr = np.zeros((tickerInfo[2].astype(int)+1,),dtype=dt2)#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 tickerInfo[1] = 0
#                 tickerInfo[2] = 500000
#                 print(np.s_[0:10])
                #print(int(tickerInfo[1]+tickerInfo[2]+1))
                #print(tickerInfo[1])
                #print(tickerInfo[1]+tickerInfo[2])
                raw_data['Quotes'].read_direct(arr,
                                               np.s_[tickerInfo[1]:int(tickerInfo[1]+tickerInfo[2]+1)],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
                                               np.s_[0:int(tickerInfo[2]+1)])#tickerInfo[2]
#                 print(arr)
#                 tempData = pd.DataFrame(arr,columns=dt2.names)


                print(ticker)
                
                #tickerInfo = QI[QIC==ticker][0]
                #pos_start.append(tickerInfo[1])
                #pos_range.append(tickerInfo[2])
                print(tickerInfo)
                #print('6 Lap time: %.3f' % ((time.time()-start)))

                # use boolean mask to slice all at once 
                mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)
                pos_start = tickerInfo[1]
                pos_range = tickerInfo[2]
                mask[pos_start : pos_start + pos_range] = True
                tempData = raw_data['Quotes'][mask]
                #print(tempData[:5])




                print('7 Lap time: %.3f' % ((time.time()-start)))
                
                # For first file and first ticker.
                if (i == 0) & (j == 0):

#                     quoteData = pd.DataFrame(tempData, columns= tempData.dtype.names)
                    quoteData = pd.DataFrame(arr, columns= dt2.names)
                    # We remove all unnecessary variables
                    unnecessaryVariables = ['NationalBBOInd',
                                            'FinraBBOInd',
                                            'FinraQuoteIndicator',
                                            'SequenceNumber',
                                            'FinraAdfMpidIndicator',
                                            'QuoteCancelCorrection',
                                            'SourceQuote',
                                            'RPI',
                                            'ShortSaleRestrictionIndicator',
                                            'LuldBBOIndicator',
                                            'SIPGeneratedMessageIdent',
                                            'NationalBBOLuldIndicator',
                                            'ParticipantTimestamp',
                                            'FinraTimestamp',
                                            'FinraQuoteIndicator',
                                            'SecurityStatusIndicator']

                    quoteData = quoteData.drop(columns=unnecessaryVariables)
                    
                    print('8 Lap time: %.3f' % ((time.time()-start)))
                    
                    
                    quoteData.loc[:,'ex'] = strList(quoteData.ex)
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding the date of the file to the dataframe.
                    quoteData['Date'] = re.split('[._]',
                                                 file)[1]
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding a more readable timestamp - TEST IT
                    quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                                file)[1],
                                                                       quoteData.utcsec))
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                        quoteData.utcsec))
                    
                    print('13 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['Hour'] = quoteData.Timestamp.dt.hour
                    quoteData['Minute'] = quoteData.Timestamp.dt.minute
                    # Adding the ticker
                    quoteData['Ticker'] = ticker

                    if (verbose) & (i==0) & (j==0):
                        print('Sneak peak of the data\n\n',quoteData.head())
                    
                    #raw_data.close()
                else:

                    # Storing the data on the following tickers in a temporary variable.

                    temp = pd.DataFrame(arr, columns= dt2.names)
                    # Removing all unnecessary variables
                    temp = temp.drop(columns=unnecessaryVariables)

                    temp.loc[:,'ex'] = strList(temp.ex)
                    temp.loc[:,'mode'] = strList(temp['mode'])

                    # Adding the date of the file to the dataframe.
                    temp['Date'] = re.split('[._]',file)[1]

                    # Adding a more readable timestamp - TEST IT
                    temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                    temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                    temp['Hour'] = temp.Timestamp.dt.hour
                    temp['Minute'] = temp.Timestamp.dt.minute

                    # Adding the ticker
                    temp['Ticker'] = ticker

                    # Adding the new data
                    quoteData = pd.concat([quoteData,temp])

    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

# Test all
- V1 and v3 seem to not reset index (see output below), but v5 does reset index. V5 is a copy of v2 with boolean masking as slicing
- V2 and V4 are not working for FMNS
- Boolean masking for all tickers (v5) at the same time is fastest (5 sec)
 - V1 is second-fastest with 13 sec
 - then V6 (boolean masking in loop) with 16 sec
 
### In order to start extracting candles within the load_data() function, we need a good way to separate data from each ticker (especially for the non-loop ticker versions)

In [174]:
timer=time.time()
data = load_data(dates, tickers, dataNeeded, path, False)
print(time.time()-timer)
data

24.8605535030365


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757321,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108,001,19,59,TSLA,706.500,5.00
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39


In [175]:
data = load_data_v1(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.000
4 Lap time: 0.000
5 Lap time: 0.799
GOOG
6 Lap time: 0.800
7 Lap time: 0.958
8 Lap time: 1.143
9 Lap time: 1.324
10 Lap time: 1.496
11 Lap time: 1.500
12 Lap time: 3.604
13 Lap time: 3.846
TSLA
6 Lap time: 3.896
7 Lap time: 4.366
The extraction time was 13.148 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39
1757325,71999034918242,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:59.034918,242,19,59,TSLA,706.500,5.00


In [170]:
# # not working for FMNS
# data = load_data_v2(dates, tickers, dataNeeded, path, False)
# data

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.816
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 0.817


TypeError: PointSelection __getitem__ only works with bool arrays

In [176]:
data = load_data_v3(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.814
GOOG
6 Lap time: 0.815
7 Lap time: 4.479
8 Lap time: 4.706
9 Lap time: 4.707
10 Lap time: 4.707
11 Lap time: 4.711
12 Lap time: 6.840
13 Lap time: 7.079
TSLA
6 Lap time: 7.128
7 Lap time: 17.592
The extraction time was 25.502 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757321,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108,001,19,59,TSLA,706.500,5.00
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39


In [172]:
# not working for FMNS
data = load_data_v4(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.803
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 0.805


TypeError: PointSelection __getitem__ only works with bool arrays

In [183]:
data = load_data_v5(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.800
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 0.801
7 Lap time: 3.287
8 Lap time: 3.942
9 Lap time: 4.595
10 Lap time: 5.231
11 Lap time: 5.246
12 Lap time: 5.452
13 Lap time: 5.453
The extraction time was 5.612 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219714,4,0,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219930,4,0,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415886,4,0,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419152,4,0,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630302,4,0,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373501,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108001,19,59,706.500,5.00
2373502,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615556,19,59,706.500,5.00
2373503,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917077,19,59,706.500,5.00
2373504,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445556,19,59,706.195,4.39


In [182]:
data = load_data_v6(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.803
GOOG
6 Lap time: 0.804
GOOG
(b'GOOG             ', 357816209, 616180)
7 Lap time: 2.188
8 Lap time: 2.378
9 Lap time: 2.563
10 Lap time: 2.731
11 Lap time: 2.735
12 Lap time: 4.837
13 Lap time: 5.096
TSLA
6 Lap time: 5.142
TSLA
(b'TSLA             ', 955832818, 1757326)
7 Lap time: 7.664
The extraction time was 16.423 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39
1757325,71999034918242,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:59.034918,242,19,59,TSLA,706.500,5.00


In [156]:
#data_range = np.concatenate([np.arange(i,j+i) for i,j in zip(pos_start,pos_range)])
pos_start, pos_range = [], []
for i in [0,1,2]: 
    pos_start.append(i)
    pos_range.append(3)
    

In [157]:
for i,j in zip(pos_start,pos_range):
    print(i,j)

0 3
1 3
2 3


In [161]:
test_zero = np.arange(10)
#test_zero[np.r_[2:5, 6:9]]

In [162]:
test_zero = np.arange(10)
selector = np.array([[0, 2], [6, 9]])

mask = np.zeros(test_zero.shape[0],dtype=bool)
for (i,j) in selector:
    mask[i:j] = True

out = test_zero[mask]
out

array([0, 1, 6, 7, 8])

In [ ]:
for (i,j) in zip(pos_start,pos_range):
    print(i,i+j)
    mask[i:i+j] = True

In [ ]:
mask

In [163]:
#data_v4 = load_data_v4(dates, tickers, dataNeeded, path, False)

In [ ]:
data_v2 = load_data_v2(dates, tickers, dataNeeded, path, False)

In [164]:
data_v2 = load_data_v2(dates, tickers, dataNeeded, path, False)

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.794
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 0.796
[0 1 2 1 2]
[(14400027066425, b'P',  0.  , 0, 100. , 6, b'R', 10401, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026882048, 99, b'', b' ')
 (14400027066425, b'P', 67.27, 1, 100. , 6, b'R', 10402, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026884608, 99, b'', b' ')
 (14400027066425, b'P', 67.27, 1,  81.1, 1, b'R', 10403, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026886144, 99, b'', b' ')
 (14400027088737, b'P', 67.27, 1,  78.2, 1, b'R', 10501, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026944512, 99, b'', b' ')
 (14400027088737, b'P', 74.6 , 1,  78.2, 1, b'R', 10503, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026946048, 99, b'', b' ')]
[(14400054219714, b'P',  980., 5,    0. , 0, b'R', 2136, b'3', b'', b'', b'', b

In [125]:
#!pip install h5py --user --upgrade

In [111]:
data_v2.head()

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,price,spread
0,14400055186285,P,414.00,1,0.0,0,R,20200501,2020-05-01 04:00:00.055186285,4,0,207.000,-414.00
1,14400055186951,P,518.93,1,0.0,0,R,20200501,2020-05-01 04:00:00.055186951,4,0,259.465,-518.93
2,14400055188470,P,518.93,1,855.0,1,R,20200501,2020-05-01 04:00:00.055188470,4,0,686.965,336.07
3,14400055239554,P,668.00,1,855.0,1,R,20200501,2020-05-01 04:00:00.055239554,4,0,761.500,187.00
4,14400055610520,P,670.00,1,855.0,1,R,20200501,2020-05-01 04:00:00.055610520,4,0,762.500,185.00


In [165]:
data_v2

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219714,4,0,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219930,4,0,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415886,4,0,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419152,4,0,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630302,4,0,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373501,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108001,19,59,706.500,5.00
2373502,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615556,19,59,706.500,5.00
2373503,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917077,19,59,706.500,5.00
2373504,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445556,19,59,706.195,4.39


In [68]:
test = pd.DataFrame(pd.to_datetime(data_v2.utcsec))
# test.loc[:,'utcsec'].dt.year = 2020

In [66]:
def vec_dt_replace(series, year=None, month=None, day=None):
    return pd.to_datetime(
        {'year': series.dt.year if year is None else year,
         'month': series.dt.month if month is None else month,
         'day': series.dt.day if day is None else day})

In [83]:
pd.to_datetime({
    'year':2020,
    'month':4,
                    'day':12})

ValueError: If using all scalar values, you must pass an index

In [84]:
pd.to_datetime('13000101', format='%Y%m%d', errors='ignore')

datetime.datetime(1300, 1, 1, 0, 0)

In [93]:
pd.Timedelta(test.loc[0,'utcsec'],unit='ns')

ValueError: Value must be Timedelta, string, integer, float, timedelta or convertible, not Timestamp

In [95]:
dates = pd.to_datetime('20200401', format='%Y%m%d', errors='ignore')
times = pd.to_timedelta(14400049177409)
datetimes  = dates + times
datetimes

Timestamp('2020-04-01 04:00:00.049177409')

In [97]:
data_v2.loc[0:10,'Date']

0     20200401
1     20200401
2     20200401
3     20200401
4     20200401
5     20200401
6     20200401
7     20200401
8     20200401
9     20200401
10    20200401
Name: Date, dtype: object

In [103]:
dates = pd.to_datetime('20200401', format='%Y%m%d', errors='ignore')
times = pd.to_timedelta(data_v2.loc[:,'utcsec'])
datetimes  = dates + times
datetimes

0         2020-04-01 04:00:00.049177409
1         2020-04-01 04:00:00.049177610
2         2020-04-01 04:00:00.049181518
3         2020-04-01 04:00:00.049181691
4         2020-04-01 04:00:00.049274621
                       ...             
1349990   2020-04-01 19:59:57.838287691
1349991   2020-04-01 19:59:59.205289502
1349992   2020-04-01 19:59:59.205301359
1349993   2020-04-01 19:59:59.205313153
1349994   2020-04-01 19:59:59.205323826
Name: utcsec, Length: 1349995, dtype: datetime64[ns]

In [99]:
dates

0    2020-04-01
1    2020-04-01
2    2020-04-01
3    2020-04-01
4    2020-04-01
5    2020-04-01
6    2020-04-01
7    2020-04-01
8    2020-04-01
9    2020-04-01
10   2020-04-01
Name: Date, dtype: datetime64[ns]

In [89]:
pd.to_datetime(data_v2.loc[0,'Date'], format='%Y%m%d', errors='ignore')+pd.Series(pd.Timedelta(ns=test.loc[0,'utcsec']))

TypeError: Invalid type <class 'pandas._libs.tslibs.timestamps.Timestamp'>. Must be int or float.

In [85]:
pd.to_datetime(data_v2.loc[0,'Date'], format='%Y%m%d', errors='ignore')


Timestamp('2020-04-01 00:00:00')

In [75]:
vec_dt_replace(test.utcsec,2020,4,2)

ValueError: If using all scalar values, you must pass an index

In [64]:
pd.to_datetime(data_v2.utcsec).dt.year = 2020

ValueError: modifications to a property of a datetimelike object are not supported. Change values on the original.

In [45]:
data_v2.utcsec.to_Timestamp()

AttributeError: 'Series' object has no attribute 'to_Timestamp'

In [50]:
data_v2.loc[0:2,'utcsec'].to_timestamp()

AttributeError: 'RangeIndex' object has no attribute 'to_timestamp'

In [48]:
pd.Timestamp(data_v2.loc[0:2,'utcsec'],unit='ns')

TypeError: Cannot convert input [0    14400049177409
1    14400049177610
2    14400049181518
Name: utcsec, dtype: uint64] of type <class 'pandas.core.series.Series'> to Timestamp

In [37]:
pd.Timestamp(2020,12,1,0,0,0,0,data_v2.loc[0,'utcsec'])

OverflowError: Python int too large to convert to C long

In [22]:
data_v3 = load_data_v3(dates, tickers, dataNeeded, path, False)

1 Lap time: 2.858
2 Lap time: 2.863
3 Lap time: 3.410
4 Lap time: 3.422
5 Lap time: 8.423
GOOG
6 Lap time: 8.993
7 Lap time: 84.574
8 Lap time: 85.288
9 Lap time: 85.290
10 Lap time: 85.291
11 Lap time: 85.303
12 Lap time: 95.556
13 Lap time: 96.839
TSLA
6 Lap time: 97.135
7 Lap time: 192.666
The extraction time was 208.103 seconds.


In [167]:
data = load_data(dates, tickers, dataNeeded, path, True)

['.ipynb_checkpoints', 'feature_engineering.ipynb', 'Old Versions', 'optimizing_dataload.ipynb', 'optimizing_dataload_FRST.ipynb', 'preprocessing.ipynb', 'Quote_vs_trade_candles.ipynb', 'SpreadInvestigation.ipynb', 'testing.hdf5', 'testing1.hdf5', 'testing_and_comparing_candlesticks.ipynb', 'testing_features.ipynb', 'testing_preprocessing_features_and_labels.ipynb']
1 ['taqquote_20200501.h5'] ['taqquote_20200501.h5']
['taqquote_20200501.h5']
##### Date range #####

Date, Min: 20200501
Date, Max: 20200501

##### Data Extraction begins #####

quote data is being extracted..

### Quote Data ###

The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

Ticker Information:  (b'GOOG             ', 357816209, 616180) 

Sneak peak of the data

            utcsec ex     bid  bidsize     ofr  ofrsize mode      Date  \
0  14400054219714  P   980.0        5     0.0        0    R  20200501   
1  14400054219930  P  1177.0        1     0.0        0    R  20200501   
2  14400054415886  P  1177.0   

In [20]:
print(dr[0],dr[600342])
print(dr[600342]-dr[0])

403024728.0 403625070.0
600342.0


In [21]:
dr[600342:600345]

array([4.03625070e+08, 1.06173525e+09, 1.06173525e+09])

In [13]:
data_v4

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,price,spread
0,14400049177409,P,963.00,1,0.00,0,R,20200401,2020-04-01 04:00:00.049177,409,4,0,481.500,-963.00
1,14400049177610,P,985.65,1,0.00,0,R,20200401,2020-04-01 04:00:00.049177,610,4,0,492.825,-985.65
2,14400049181518,P,999.00,1,0.00,0,R,20200401,2020-04-01 04:00:00.049181,518,4,0,499.500,-999.00
3,14400049181691,P,1018.00,1,0.00,0,R,20200401,2020-04-01 04:00:00.049181,691,4,0,509.000,-1018.00
4,14400049274621,P,1018.00,1,1188.88,3,R,20200401,2020-04-01 04:00:00.049274,621,4,0,1103.440,170.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600338,71940158291291,Q,1105.00,1,1112.65,1,R,20200401,2020-04-01 19:59:00.158291,291,19,59,1108.825,7.65
600339,71943088451424,Q,1105.00,1,1113.00,1,R,20200401,2020-04-01 19:59:03.088451,424,19,59,1109.000,8.00
600340,71944334686239,K,1110.04,1,1114.42,1,R,20200401,2020-04-01 19:59:04.334686,239,19,59,1112.230,4.38
600341,71947345326862,K,1110.04,1,1114.42,1,R,20200401,2020-04-01 19:59:07.345326,862,19,59,1112.230,4.38


In [7]:
data = load_data(dates, tickers, dataNeeded, path, False)

1 Lap time: 3.077
2 Lap time: 3.081
3 Lap time: 3.766
4 Lap time: 3.781
5 Lap time: 7.923
GOOG
6 Lap time: 7.925
7 Lap time: 66.387
8 Lap time: 66.992
9 Lap time: 67.613
10 Lap time: 68.237
11 Lap time: 68.248
12 Lap time: 75.816
13 Lap time: 76.742
MSFT
6 Lap time: 76.990
7 Lap time: 623.323
The extraction time was 742.092 seconds.


In [24]:
data[data.Ticker=='MSFT']

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400006484836,Q,96.00,2,0.00,0,R,20200401,2020-04-01 04:00:00.006484,836,4,0,MSFT,48.000,-96.00
1,14400039870729,P,145.09,55,0.00,0,R,20200401,2020-04-01 04:00:00.039870,729,4,0,MSFT,72.545,-145.09
2,14400039878245,P,145.09,55,176.80,1,R,20200401,2020-04-01 04:00:00.039878,245,4,0,MSFT,160.945,31.71
3,14400040037162,P,150.00,1,176.80,1,R,20200401,2020-04-01 04:00:00.040037,162,4,0,MSFT,163.400,26.80
4,14400040041937,P,150.00,1,161.52,3,R,20200401,2020-04-01 04:00:00.040041,937,4,0,MSFT,155.760,11.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5607203,71993763255970,K,153.41,2,153.59,2,R,20200401,2020-04-01 19:59:53.763255,970,19,59,MSFT,153.500,0.18
5607204,71996762822526,K,153.58,2,153.59,2,R,20200401,2020-04-01 19:59:56.762822,526,19,59,MSFT,153.585,0.01
5607205,71998862697002,K,153.58,2,153.59,1,R,20200401,2020-04-01 19:59:58.862697,002,19,59,MSFT,153.585,0.01
5607206,71999986468655,K,153.58,1,153.59,1,R,20200401,2020-04-01 19:59:59.986468,655,19,59,MSFT,153.585,0.01


In [9]:
data_v2 = load_data_v2(dates, tickers, dataNeeded, path, False)

1 Lap time: 3.302
2 Lap time: 3.307
3 Lap time: 4.066
4 Lap time: 4.083
5 Lap time: 8.877
GOOG
(b'GOOG             ', 403024728, 600343)
MSFT
(b'MSFT             ', 659389624, 5607207)
6 Lap time: 8.884
7 Lap time: 769.333
8 Lap time: 777.292
9 Lap time: 784.501
10 Lap time: 791.502
11 Lap time: 791.605
12 Lap time: 880.095
13 Lap time: 890.045
The extraction time was 892.524 seconds.


In [10]:
data_v2

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,price,spread
0,14400049177409,P,963.00,1,0.00,0,R,20200401,2020-04-01 04:00:00.049177,409,4,0,481.500,-963.00
1,14400049177610,P,985.65,1,0.00,0,R,20200401,2020-04-01 04:00:00.049177,610,4,0,492.825,-985.65
2,14400049181518,P,999.00,1,0.00,0,R,20200401,2020-04-01 04:00:00.049181,518,4,0,499.500,-999.00
3,14400049181691,P,1018.00,1,0.00,0,R,20200401,2020-04-01 04:00:00.049181,691,4,0,509.000,-1018.00
4,14400049274621,P,1018.00,1,1188.88,3,R,20200401,2020-04-01 04:00:00.049274,621,4,0,1103.440,170.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207545,71990849475938,K,153.41,2,153.59,2,R,20200401,2020-04-01 19:59:50.849475,938,19,59,153.500,0.18
6207546,71993763255970,K,153.41,2,153.59,2,R,20200401,2020-04-01 19:59:53.763255,970,19,59,153.500,0.18
6207547,71996762822526,K,153.58,2,153.59,2,R,20200401,2020-04-01 19:59:56.762822,526,19,59,153.585,0.01
6207548,71998862697002,K,153.58,2,153.59,1,R,20200401,2020-04-01 19:59:58.862697,002,19,59,153.585,0.01


In [28]:
data_v2.loc[600343:,:]

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,price,spread
600343,14400006484836,Q,96.00,2,0.00,0,R,20200401,2020-04-01 04:00:00.006484,836,4,0,48.000,-96.00
600344,14400039870729,P,145.09,55,0.00,0,R,20200401,2020-04-01 04:00:00.039870,729,4,0,72.545,-145.09
600345,14400039878245,P,145.09,55,176.80,1,R,20200401,2020-04-01 04:00:00.039878,245,4,0,160.945,31.71
600346,14400040037162,P,150.00,1,176.80,1,R,20200401,2020-04-01 04:00:00.040037,162,4,0,163.400,26.80
600347,14400040041937,P,150.00,1,161.52,3,R,20200401,2020-04-01 04:00:00.040041,937,4,0,155.760,11.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207545,71990849475938,K,153.41,2,153.59,2,R,20200401,2020-04-01 19:59:50.849475,938,19,59,153.500,0.18
6207546,71993763255970,K,153.41,2,153.59,2,R,20200401,2020-04-01 19:59:53.763255,970,19,59,153.500,0.18
6207547,71996762822526,K,153.58,2,153.59,2,R,20200401,2020-04-01 19:59:56.762822,526,19,59,153.585,0.01
6207548,71998862697002,K,153.58,2,153.59,1,R,20200401,2020-04-01 19:59:58.862697,002,19,59,153.585,0.01


In [82]:
npRand = np.random.randint(0,10,100)
# tf = tempfile.TemporaryFile()
f = h5py.File('testing1.hdf5','w')

In [264]:
f['dset11'].dtype

dtype([('name', '<f8'), ('grade_1', '<f8'), ('grade_2', '<f8'), ('grade_3', '<f8'), ('grade_4', '<f8')])

In [260]:
f['dset11']

AttributeError: 'Dataset' object has no attribute 'keys'

In [142]:
npTuple = [np.void((i,np.random.randint(0,10,dtype='u8'),
            np.random.randint(0,10,dtype='u8'),
            np.random.randint(0,10,dtype='u8'))) for i in np.arange(0,10)]

In [179]:
type(npTuple[0])

numpy.ndarray

In [109]:
npTuple = np.void([(i,np.random.randint(0,10,dtype='i8'),
            np.random.randint(0,10,dtype='i8'),
            np.random.randint(0,10,dtype='i8')) for i in np.arange(0,100)])

In [144]:
npTuple

[array([b'\x00\x00\x00\x00\x00\x00\x00\x00',
        b'\x01\x00\x00\x00\x00\x00\x00\x00',
        b'\x01\x00\x00\x00\x00\x00\x00\x00',
        b'\x06\x00\x00\x00\x00\x00\x00\x00'], dtype='|V8'),
 array([b'\x01\x00\x00\x00\x00\x00\x00\x00',
        b'\x01\x00\x00\x00\x00\x00\x00\x00',
        b'\x02\x00\x00\x00\x00\x00\x00\x00',
        b'\x03\x00\x00\x00\x00\x00\x00\x00'], dtype='|V8'),
 array([b'\x02\x00\x00\x00\x00\x00\x00\x00',
        b'\x08\x00\x00\x00\x00\x00\x00\x00',
        b'\x08\x00\x00\x00\x00\x00\x00\x00',
        b'\x02\x00\x00\x00\x00\x00\x00\x00'], dtype='|V8'),
 array([b'\x03\x00\x00\x00\x00\x00\x00\x00',
        b'\x00\x00\x00\x00\x00\x00\x00\x00',
        b'\x04\x00\x00\x00\x00\x00\x00\x00',
        b'\x01\x00\x00\x00\x00\x00\x00\x00'], dtype='|V8'),
 array([b'\x04\x00\x00\x00\x00\x00\x00\x00',
        b'\x02\x00\x00\x00\x00\x00\x00\x00',
        b'\x05\x00\x00\x00\x00\x00\x00\x00',
        b'\x04\x00\x00\x00\x00\x00\x00\x00'], dtype='|V8'),
 array([b'\x05\x00\x00\x0

In [146]:
dset = f.create_dataset("dset6", data=npTuple)#, dtype='int64'
arr = np.zeros((100,))#, dtype='int32'
print(dset.read_direct(arr, np.s_[0:10], np.s_[50:60]))

TypeError: Can't broadcast (10, 4) -> (10,)

In [121]:
arr

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [79]:
f.close()

In [14]:
# dt = np.dtype([('name', np.str_, 16),
#                ('grade_1', np.float64),
#                 ('grade_2', np.float64),
#                  ('grade_3', np.float64),
#                 ('grade_4', np.float64)])
dt1 = np.dtype([('name', np.float),
               ('grade_1', np.float64),
                ('grade_2', np.float64),
                 ('grade_3', np.float64),
                ('grade_4', np.float64)])
x = np.array([(0.9, 8.0, 7.0,1.022,2.3),
              (1.9, 6.0, 7.0,4.5,4.5),
              (3.9, 6.0, 7.0,4.5,4.5),
              (4.9, 6.0, 7.0,4.5,4.5),
              (5.9, 6.0, 7.0,4.5,4.5),
              (6.9, 6.0, 7.0,4.5,4.5),
              (7.9, 6.0, 7.0,4.5,4.5)], dtype=dt1)

In [15]:
type(x)

numpy.ndarray

In [16]:
type(x[0])

numpy.void

In [17]:
test = np.zeros((1,),dtype=dt1)

In [18]:
test[0] = x[0]

In [19]:
test[0]

(0.9, 8., 7., 1.022, 2.3)

In [20]:
x[0]=(0,0,0,0,0)

In [21]:
y = np.concatenate([[0,1],[5]])
print(y)

[0 1 5]


In [25]:
print(np.r_[0:2,5:10])

[0 1 5 6 7 8 9]


In [22]:
x[np.r_[0:2,5:6]]

array([(0. , 0., 0., 0. , 0. ), (1.9, 6., 7., 4.5, 4.5),
       (6.9, 6., 7., 4.5, 4.5)],
      dtype=[('name', '<f8'), ('grade_1', '<f8'), ('grade_2', '<f8'), ('grade_3', '<f8'), ('grade_4', '<f8')])

In [23]:
x[np.r_[y]]

array([(0. , 0., 0., 0. , 0. ), (1.9, 6., 7., 4.5, 4.5),
       (6.9, 6., 7., 4.5, 4.5)],
      dtype=[('name', '<f8'), ('grade_1', '<f8'), ('grade_2', '<f8'), ('grade_3', '<f8'), ('grade_4', '<f8')])

In [24]:
x[[0,1,5]]

array([(0. , 0., 0., 0. , 0. ), (1.9, 6., 7., 4.5, 4.5),
       (6.9, 6., 7., 4.5, 4.5)],
      dtype=[('name', '<f8'), ('grade_1', '<f8'), ('grade_2', '<f8'), ('grade_3', '<f8'), ('grade_4', '<f8')])

In [35]:
x.shape

(7,)

In [45]:
x[lambda x: ]

array([(0.9, 8., 7., 1.022, 2.3), (1.9, 6., 7., 4.5  , 4.5),
       (1.9, 6., 7., 4.5  , 4.5), (1.9, 6., 7., 4.5  , 4.5),
       (1.9, 6., 7., 4.5  , 4.5), (1.9, 6., 7., 4.5  , 4.5),
       (1.9, 6., 7., 4.5  , 4.5)],
      dtype=[('name', '<f8'), ('grade_1', '<f8'), ('grade_2', '<f8'), ('grade_3', '<f8'), ('grade_4', '<f8')])

In [43]:
x[x[0]<1]

TypeError: '<' not supported between instances of 'numpy.ndarray' and 'int'

In [250]:
dset = f.create_dataset("dset11", data=x)#, dtype='int64'


None


In [ ]:
arr = np.zeros((5,),dtype=dt1)#, dtype='int32'
print(dset.read_direct(arr, np.s_[0:5], np.s_[0:5]))

In [251]:
arr

array([(0. , 0., 0., 0. , 0. ), (1.9, 6., 7., 4.5, 4.5),
       (1.9, 6., 7., 4.5, 4.5), (1.9, 6., 7., 4.5, 4.5),
       (1.9, 6., 7., 4.5, 4.5)],
      dtype=[('name', '<f8'), ('grade_1', '<f8'), ('grade_2', '<f8'), ('grade_3', '<f8'), ('grade_4', '<f8')])

In [176]:
from pandas.io.pytables import Term

In [233]:
start = time.time()
# Pandas hdf5
store = pd.HDFStore(path+'/'+'taqquote_20200401.h5', 'r') #.get('Quotes')
print(store is None)
# print(store.select(start=0, stop=2, chunksize=1))
# print(store.select(key='Quotes', start=0, stop=1,columns=['utcsec', 'ex', 'bid', 'bidsize', 'ofr', 'ofrsize', 'mode',
#        'SequenceNumber', 'NationalBBOInd', 'FinraBBOInd',
#        'FinraAdfMpidIndicator', 'QuoteCancelCorrection'
#        ]).columns)#, chunksize=1
# Term('columns', '=', ['utcsec','ex'])
meta = store.select('Quotes', start=1, stop=1)
print(store.select('Quotes', columns=meta.columns[0],start=0, stop=4).columns)#, chunksize=1
# hejmeddig = store.select(key='Quotes',
#                          start=10000000,
#                          stop=10500000,columns=['utcsec',
#                                                 'ex',
#                                                 'bid',
#                                                 'ofr'])#, chunksize=1



end = time.time()
print('Processing time: %.3f'%(end-start))

False
Index(['utcsec', 'ex', 'bid', 'bidsize', 'ofr', 'ofrsize', 'mode',
       'SequenceNumber', 'NationalBBOInd', 'FinraBBOInd',
       'FinraAdfMpidIndicator', 'QuoteCancelCorrection', 'SourceQuote', 'RPI',
       'ShortSaleRestrictionIndicator', 'LuldBBOIndicator',
       'SIPGeneratedMessageIdent', 'NationalBBOLuldIndicator',
       'ParticipantTimestamp', 'FinraTimestamp', 'FinraQuoteIndicator',
       'SecurityStatusIndicator'],
      dtype='object')
Processing time: 0.495


In [234]:
meta.index


Int64Index([], dtype='int64')

In [236]:
meta.columns

Index(['utcsec', 'ex', 'bid', 'bidsize', 'ofr', 'ofrsize', 'mode',
       'SequenceNumber', 'NationalBBOInd', 'FinraBBOInd',
       'FinraAdfMpidIndicator', 'QuoteCancelCorrection', 'SourceQuote', 'RPI',
       'ShortSaleRestrictionIndicator', 'LuldBBOIndicator',
       'SIPGeneratedMessageIdent', 'NationalBBOLuldIndicator',
       'ParticipantTimestamp', 'FinraTimestamp', 'FinraQuoteIndicator',
       'SecurityStatusIndicator'],
      dtype='object')

In [237]:
meta.dtypes

utcsec                            uint64
ex                                object
bid                              float64
bidsize                           uint32
ofr                              float64
ofrsize                           uint32
mode                              object
SequenceNumber                    uint64
NationalBBOInd                    object
FinraBBOInd                       object
FinraAdfMpidIndicator             object
QuoteCancelCorrection             object
SourceQuote                       object
RPI                               object
ShortSaleRestrictionIndicator     object
LuldBBOIndicator                  object
SIPGeneratedMessageIdent          object
NationalBBOLuldIndicator          object
ParticipantTimestamp              uint64
FinraTimestamp                    uint64
FinraQuoteIndicator               object
SecurityStatusIndicator           object
dtype: object

In [241]:
dtDict = [(i,j) for i,j in zip(meta.columns,meta.dtypes)]
dt2 = np.dtype(dtDict)

In [271]:
dt2.names

('utcsec',
 'ex',
 'bid',
 'bidsize',
 'ofr',
 'ofrsize',
 'mode',
 'SequenceNumber',
 'NationalBBOInd',
 'FinraBBOInd',
 'FinraAdfMpidIndicator',
 'QuoteCancelCorrection',
 'SourceQuote',
 'RPI',
 'ShortSaleRestrictionIndicator',
 'LuldBBOIndicator',
 'SIPGeneratedMessageIdent',
 'NationalBBOLuldIndicator',
 'ParticipantTimestamp',
 'FinraTimestamp',
 'FinraQuoteIndicator',
 'SecurityStatusIndicator')

In [174]:
[ Term('index', '>', Timestamp('20010105')), 
                     Term('columns', '=', ['A','B']) ])

[]

In [165]:
store.get('Quotes').head()

MemoryError: Unable to allocate 86.9 GiB for an array with shape (1313723367,) and data type [('utcsec', '<u8'), ('ex', 'S1'), ('bid', '<f8'), ('bidsize', '<u4'), ('ofr', '<f8'), ('ofrsize', '<u4'), ('mode', 'S1'), ('SequenceNumber', '<u8'), ('NationalBBOInd', 'S1'), ('FinraBBOInd', 'S1'), ('FinraAdfMpidIndicator', 'S1'), ('QuoteCancelCorrection', 'S1'), ('SourceQuote', 'S1'), ('RPI', 'S1'), ('ShortSaleRestrictionIndicator', 'S1'), ('LuldBBOIndicator', 'S1'), ('SIPGeneratedMessageIdent', 'S1'), ('NationalBBOLuldIndicator', 'S1'), ('ParticipantTimestamp', '<u8'), ('FinraTimestamp', '<u8'), ('FinraQuoteIndicator', 'S1'), ('SecurityStatusIndicator', 'S2')]

In [162]:
hejmeddig.head()

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,SequenceNumber,NationalBBOInd,FinraBBOInd,...,SourceQuote,RPI,ShortSaleRestrictionIndicator,LuldBBOIndicator,SIPGeneratedMessageIdent,NationalBBOLuldIndicator,ParticipantTimestamp,FinraTimestamp,FinraQuoteIndicator,SecurityStatusIndicator
0,52224781525616,P,19.68,2,19.75,20,R,3085672201,U,,...,C,,,,,A,52224781442816,99,,
1,52224781605812,T,19.68,2,20.00,13,R,3085672301,U,,...,C,,,,,A,52224781097034,99,,
2,52224781637538,T,19.68,2,19.76,20,R,3085672401,A,,...,C,,,,,A,52224781142665,99,,
3,52227190475701,P,19.68,2,19.74,20,R,3086090301,U,,...,C,,,,,A,52227190359552,99,,
4,52227190790740,T,19.68,2,20.02,13,R,3086090401,U,,...,C,,,,,A,52227190240056,99,,


In [7]:
cProfile.run('load_data(dates, tickers, dataNeeded, path, False)')

         17011386 function calls (17001419 primitive calls) in 104.065 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(amin)
       16    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(append)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(argsort)
       17    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(atleast_2d)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(bincount)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(can_cast)
       42    0.000    0.000    0.105    0.003 <__array_function__ internals>:2(concatenate)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(copyto)
        1    0.000    0.0

        1    0.000    0.000    0.000    0.000 __init__.py:1216(getLogger)
        1    0.000    0.000    0.000    0.000 __init__.py:1267(_fixupParents)
        1    0.000    0.000    0.000    0.000 __init__.py:1336(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:1368(info)
        3    0.000    0.000    0.000    0.000 __init__.py:156(maybe_upcast_for_op)
        1    0.000    0.000    0.000    0.000 __init__.py:1600(getEffectiveLevel)
        1    0.000    0.000    0.000    0.000 __init__.py:1614(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:187(_checkLevel)
        1    0.000    0.000    0.000    0.000 __init__.py:1925(getLogger)
        2    0.000    0.000    0.000    0.000 __init__.py:212(_acquireLock)
        1    0.000    0.000    0.017    0.017 __init__.py:22(<module>)
        2    0.000    0.000    0.000    0.000 __init__.py:221(_releaseLock)
        3    0.000    0.000    0.000    0.000 __init__.py:424(_align_method_SERIES)
        

        8    0.000    0.000    0.000    0.000 base.py:2922(is_int)
        1    0.000    0.000    0.000    0.000 base.py:3084(_can_reindex)
        1    0.000    0.000    0.000    0.000 base.py:3101(reindex)
       37    0.000    0.000    0.000    0.000 base.py:3639(values)
        5    0.000    0.000    0.000    0.000 base.py:3671(_values)
        8    0.000    0.000    0.000    0.000 base.py:3701(_internal_get_values)
        8    0.000    0.000    0.006    0.001 base.py:3810(_coerce_scalar_to_index)
       64    0.001    0.000    0.001    0.000 base.py:3898(__contains__)
       51    0.000    0.000    0.001    0.000 base.py:3912(__getitem__)
       10    0.000    0.000    0.001    0.000 base.py:3950(_can_hold_identifiers_and_holds_name)
        5    0.000    0.000    0.000    0.000 base.py:4047(equals)
        2    0.000    0.000    0.000    0.000 base.py:4077(identical)
        2    0.000    0.000    0.000    0.000 base.py:4092(<genexpr>)
       22    0.000    0.000    0.000    0.0

       13    0.000    0.000    0.000    0.000 common.py:685(is_dtype_equal)
       32    0.000    0.000    0.000    0.000 common.py:775(is_integer_dtype)
       36    0.000    0.000    0.000    0.000 common.py:830(is_signed_integer_dtype)
       34    0.000    0.000    0.000    0.000 common.py:887(is_unsigned_integer_dtype)
       96    0.000    0.000    0.002    0.000 common.py:987(is_datetime64_any_dtype)
        6    0.000    0.000    0.000    0.000 common.py:99(is_bool_indexer)
        1    0.000    0.000    0.000    0.000 compat.py:103(filename_encode)
        1    0.000    0.000    0.000    0.000 config.py:101(_get_option)
        1    0.000    0.000    0.000    0.000 config.py:230(__call__)
        1    0.000    0.000    0.000    0.000 config.py:533(_select_options)
        1    0.000    0.000    0.000    0.000 config.py:551(_get_root)
        2    0.000    0.000    0.000    0.000 config.py:566(_get_deprecated_option)
        1    0.000    0.000    0.000    0.000 config.py:594(_

        1    0.000    0.000    0.000    0.000 generic.py:11343(logical_func)
        1    0.000    0.000    0.000    0.000 generic.py:1468(__invert__)
       26    0.000    0.000    0.000    0.000 generic.py:190(__init__)
        1    0.000    0.000    0.000    0.000 generic.py:219(_init_mgr)
        8    0.000    0.000    0.000    0.000 generic.py:238(attrs)
        4    0.000    0.000    0.000    0.000 generic.py:255(_validate_dtype)
       12    0.000    0.000    0.000    0.000 generic.py:3227(_set_as_cached)
       15    0.000    0.000    0.000    0.000 generic.py:3294(_clear_item_cache)
        3    0.000    0.000    0.000    0.000 generic.py:356(_construct_axes_from_arguments)
       22    0.000    0.000    0.003    0.000 generic.py:3581(_get_item_cache)
       12    0.000    0.000    0.036    0.003 generic.py:3623(_set_item)
       12    0.000    0.000    0.000    0.000 generic.py:3656(_check_setitem_copy)
        3    0.000    0.000    0.000    0.000 generic.py:381(<dictcomp>)


        1    0.000    0.000    0.000    0.000 necompiler.py:80(ASTNode)
        3    0.000    0.000    0.000    0.000 necompiler.py:821(<listcomp>)
       29    0.000    0.000    0.000    0.000 necompiler.py:94(__init__)
        3    0.000    0.000    0.000    0.000 ntpath.py:122(splitdrive)
        1    0.000    0.000    0.000    0.000 ntpath.py:178(split)
        1    0.000    0.000    0.000    0.000 ntpath.py:219(dirname)
        1    0.000    0.000    0.000    0.000 ntpath.py:34(_get_bothseps)
        1    0.000    0.000    0.000    0.000 ntpath.py:75(join)
       17    0.000    0.000    0.000    0.000 numeric.py:155(is_all_dates)
        1    0.000    0.000    0.000    0.000 numeric.py:159(ones)
        4    0.000    0.000    0.000    0.000 numeric.py:249(inferred_type)
        2    0.000    0.000    0.000    0.000 numeric.py:274(_assert_safe_casting)
        3    0.000    0.000    0.000    0.000 numeric.py:283(full)
        2    0.000    0.000    0.008    0.004 numeric.py:53(__ne

       27    0.000    0.000    0.000    0.000 {built-in method from_bytes}
        1    0.000    0.000    0.000    0.000 {built-in method h5py.h5.get_config}
     8871    0.724    0.000    0.724    0.000 {built-in method h5py.h5t.py_create}
        9    0.001    0.000    0.001    0.000 {built-in method marshal.loads}
        1    0.000    0.000    0.000    0.000 {built-in method nt._isdir}
       34    0.000    0.000    0.000    0.000 {built-in method nt.fspath}
        1    0.000    0.000    0.000    0.000 {built-in method nt.getcwd}
        3    2.502    0.834    2.502    0.834 {built-in method nt.listdir}
       37    0.005    0.000    0.005    0.000 {built-in method nt.stat}
        1    0.000    0.000    0.000    0.000 {built-in method numexpr.interpreter._set_num_threads}
        1    0.000    0.000    0.000    0.000 {built-in method numexpr.interpreter._set_vml_num_threads}
       33    0.004    0.000    0.004    0.000 {built-in method numpy.arange}
  281/264    0.314    0.001  

       21    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_integer}
      121    0.000    0.000    0.002    0.000 {pandas._libs.lib.is_list_like}
       99    0.000    0.000    0.001    0.000 {pandas._libs.lib.is_scalar}
       25    0.000    0.000    0.000    0.000 {pandas._libs.lib.item_from_zerodim}
        3    0.004    0.001    0.004    0.001 {pandas._libs.lib.maybe_convert_objects}
       16    0.000    0.000    0.000    0.000 {pandas._libs.lib.values_from_object}
        2    0.230    0.115    0.230    0.115 {pandas._libs.tslibs.fields.get_date_field}




## Data Cleaning, Feature Engineering & Pre Processing

# Implemented technical features

A library: https://technical-analysis-library-in-python.readthedocs.io/en/latest/

### Features used in the literature:

* Stochastic K
* Stochastic D
* Slow Stochastic D
* Momentum/difference
* ROC
* Williams % R
* A/D Oscillator
* Disparity 5
* Disparity 10
* Price Oscillator - (detrended)
* Commodity Channel Index
* RSI

Formulas: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=876544

* Moving Average
* Exponential Moving Average
* True Range - (Average)

Formulas: https://www.sciencedirect.com/science/article/pii/S0957417407001819?via%3Dihub

#### Other Technical Features
* Moving Average Convergence Divergence (MACD)

**Non-classical technical features** - **NOT IMPLEMENTED**

* Bid/Ask prices of top of book
* Spread and mid price based on top og book
* Price derivatives

Formulas: https://www.tandfonline.com/doi/full/10.1080/14697688.2015.1032546?instName=UCL+%28University+College+London%29

In [15]:
########### Clean data ###########
    
DATA_SAMPLE = 'full' # or 'stable'

if DATA_SAMPLE == 'stable':
    # P1 is used for keeping data within [9.5, 16]
    cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])
elif DATA_SAMPLE == 'full':
    # P1_2 is used for keeping data within [9, 16.5]
    cleanedData = HFDataCleaning(['P1_2','p2', 'q2', 'p3'],quoteData,'quote',['q'])#'t1',tradeData # q2, quotedate
    
########### Construct Candles ################
# candles = candleCreateNP_vect_final(cleanedData
#                          ,1)

candles = candleCreateNP_vect_final(data = cleanedData,
                                       step = 1,
                                        verbose=False,
                                        fillHoles=True,
                                        sample='stable',
                                        numpied=True)

########### Generate Features ################

n_feature_lags = 0
features = generateFeatures_final(data = candles, 
                                  listOfFeatures = [
                                                    'pastobs',
                                                    'stok',
                                                    'stod',
                                                    'sstod',
                                                    'wilr',
                                                    'roc',
                                                    'rsi',
                                                    'atr',
                                                    'cci',
                                                    'dpo',
                                                    'sma',
                                                    'ema',
                                                    'macd',
                                                    'dis5',
                                                    'dis10',
                                                   ], 
                                   feature_lags = n_feature_lags)

########### Generate Labels ################

n_classes = 3

labels = extract_labels(data = candles, classes = n_classes, group_style = 'equal')

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y = align_features_and_labels(candles = candles, 
                                 prediction_horizon = 1, 
                                 features = features, 
                                 n_feature_lags = n_feature_lags, 
                                 n_classes = n_classes, # 5,
                                 safe_burn_in = False, 
                                 data_sample = 'full')